# Классификация изображений

### ЧТО НУЖНО СДЕЛАТЬ?

1. Изучить данные.
2. Построить baseline решения под руководством автора модуля.
3. Ознакомиться с критериями оценивания проекта.
4. На основе baseline и данных в нём рекомендаций повысить точность модели.
5. Загрузить итоговое решение на kaggle и добавить описание проекта на git.

In [ ]:
!nvidia-smi

In [ ]:
# Загружаем нужные пакеты
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import zipfile
import csv
import sys
import os


import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import *

from sklearn.model_selection import train_test_split, StratifiedKFold

import PIL
from PIL import ImageOps, ImageFilter
# увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
# графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

print(os.listdir("../input"))
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

**Работаем с Tensorflow v2**

In [ ]:
!pip freeze > requirements.txt

# Основные настройки

In [ ]:
# В setup выносим основные настройки: так удобнее их перебирать в дальнейшем.

EPOCHS = 8  # эпох на обучение
BATCH_SIZE = 8  # уменьшаем batch если сеть большая, иначе не влезет в память на GPU
LR = 1e-3
VAL_SPLIT = 0.15  # сколько данных выделяем на тест = 15%

CLASS_NUM = 10  # количество классов в нашей задаче
IMG_SIZE = 224  # какого размера подаем изображения в сеть
IMG_CHANNELS = 3   # у RGB 3 канала
input_shape = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

DATA_PATH = '../input/'
PATH = "../working/car/"  # рабочая директория

In [ ]:
# Устаналиваем конкретное значение random seed для воспроизводимости
os.makedirs(PATH, exist_ok=False)

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
PYTHONHASHSEED = 0

# EDA / Анализ данных

In [ ]:
train_df = pd.read_csv(DATA_PATH+"train.csv")
sample_submission = pd.read_csv(DATA_PATH+"sample-submission.csv")
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df.Category.value_counts()
# распределение классов достаточно равномерное - это хорошо

In [ ]:
print('Распаковываем картинки')
# Will unzip the files so that you can see them..
for data_zip in ['train.zip', 'test.zip']:
    with zipfile.ZipFile("../input/"+data_zip, "r") as z:
        z.extractall(PATH)

print(os.listdir(PATH))

In [ ]:
print('Пример картинок (random sample)')
plt.figure(figsize=(12, 8))

random_image = train_df.sample(n=9)
random_image_paths = random_image['Id'].values
random_image_cat = random_image['Category'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(PATH+f'train/{random_image_cat[index]}/{path}')
    plt.subplot(3, 3, index+1)
    plt.imshow(im)
    plt.title('Class: '+str(random_image_cat[index]))
    plt.axis('off')
plt.show()

Посмотрим на примеры картинок и их размеры чтоб понимать как их лучше обработать и сжимать.

In [ ]:
image = PIL.Image.open(PATH+'/train/0/100380.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

### Уже догадываетесь, что означают классы?

# Подготовка данных

### Аугментация данных

In [ ]:
# Вы помните, что аугментация данных важна, когда мы работаем с небольшим датасетом. Это как раз наш случай.
# Чтобы лучше понять работу параметров, попробуйте их изменить. К какому результату это приведет?
# Официальная документация: https://keras.io/preprocessing/image/

# Сначала поменяем параметры генератора изображений
"""train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=5,  # увеличим диапазон вращения с 5 до 50
    # zoom_range=[0.75,1.25], # добавим вариации по приближению
    # brightness_range=[0.5, 1.5], # и яркости
    width_shift_range=0.1,
    height_shift_range=0.1,
    validation_split=VAL_SPLIT,  # set validation split
    horizontal_flip=False)

test_datagen = ImageDataGenerator(rescale=1. / 255)"""

# Рекомендация Подключите более продвинутые библиотеки аугментации изображений
# (например: albumentations или imgaug, для них есть специальные "обертки" под Keras,
# например: https://github.com/mjkvaak/ImageDataAugmentor)

In [ ]:
!pip install albumentations
!pip install git+https: // github.com/mjkvaak/ImageDataAugmentor

In [ ]:
from ImageDataAugmentor.image_data_augmentor import *
import albumentations as A

AUGMENTATIONS = A.Compose([
    A.GaussianBlur(p=0.05),
    A.RandomBrightness(limit=0.2, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.0625,
                       scale_limit=0.01,
                       interpolation=1,
                       border_mode=4,
                       rotate_limit=20,
                       p=.75),
    A.OneOf([
        A.CenterCrop(height=224, width=200),
        A.CenterCrop(height=200, width=224)], p=0.5),
    A.OneOf([
        A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
        A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)], p=0.5),
    A.HorizontalFlip(p=0.5),
    A.HueSaturationValue(p=0.5),
    A.RGBShift(p=0.5),
    A.FancyPCA(alpha=0.1, always_apply=False, p=0.5),
    A.Resize(IMG_SIZE, IMG_SIZE)
])

train_datagen = ImageDataAugmentor(rescale=1./255,
                                   augment=AUGMENTATIONS,
                                   seed=RANDOM_SEED,
                                   validation_split=VAL_SPLIT
                                   )

test_datagen = ImageDataAugmentor(rescale=1. / 255)

### Генерация данных

In [ ]:
# Завернем наши данные в генератор:

train_generator = train_datagen.flow_from_directory(
    PATH+'train/',      # директория где расположены папки с картинками
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training')  # set as training data

test_generator = train_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation')  # set as validation data

test_sub_generator = test_datagen.flow_from_dataframe(
    dataframe=sample_submission,
    directory=PATH+'test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,)

# Построение модели

In [ ]:
!pip install -q efficientnet

In [ ]:
import efficientnet.tfkeras as efn

In [ ]:
# Загружаем предобученную сеть Xception:
#base_model = Xception(weights='imagenet', include_top=False, input_shape = input_shape)

# EfficientNetB6
base_model = efn.EfficientNetB6(
    weights='imagenet', include_top=False, input_shape=input_shape)

# "заморозим" веса модели
base_model.trainable = False

In [ ]:
# Посмотрим структуру базовой модели
base_model.summary()

In [ ]:
from keras.models import Sequential
# Добавим "голову"
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu', bias_regularizer=l2(
    1e-4), activity_regularizer=l2(1e-5)))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(CLASS_NUM, activation='softmax'))

In [ ]:
# Посмотрим структуру полученной модели
base_model.summary()

## Обучение модели

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])

## Настроим CallBack

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
checkpoint = ModelCheckpoint('best_model.hdf5',
                             monitor=['val_accuracy'],
                             verbose=1,
                             mode='max')

earlystop = EarlyStopping(monitor='val_accuracy',
                          patience=3,
                          restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.25,
                              patience=2,
                              min_lr=0.0000001,
                              verbose=1,
                              mode='auto')

callbacks_list = [checkpoint, earlystop, reduce_lr]

In [ ]:
def plot_history(history):
    """функция для построеия графиков"""
    plt.figure(figsize=(10, 5))
    # plt.style.use('dark_background')
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'b', label='Training acc')
    plt.plot(epochs, val_acc, 'g', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()

    # plt.figure()
    plt.figure(figsize=(10, 5))
    # plt.style.use('dark_background')
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'g', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

Обучаем:

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples//test_generator.batch_size,
    epochs=EPOCHS,
    callbacks=callbacks_list
)

# сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)
model.save('../working/model_last.hdf5')

In [ ]:
scores = model.evaluate_generator(
    test_generator, steps=len(test_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

В Итоге точность нашей модели Xception с полным переобучением составила 93%. 
Учитывая что классов 10 - это Очень хороший результат!

Точность модели EficientNetB6 с замороженными слоями базовой модели составила более 66%, что не плохо.

Base_line Accuracy: 92.06%

In [ ]:
# Посмотрим графики обучения:
plot_history(history)

### Разморозим половину базовой модели

In [ ]:
base_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = len(base_model.layers)//2

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

LR = 1e-4  # уменьшаем LR
model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])

# Загружаем веса для "головы"
model.load_weights('../working/model_last.hdf5')

# Обучаем
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples//test_generator.batch_size,
    epochs=EPOCHS,
    callbacks=callbacks_list
)

# сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)
model.save('../working/model_last.hdf5')

scores = model.evaluate_generator(
    test_generator, steps=len(test_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

plot_history(history)

Переобучив половину слоев базовой модели удалось превысить точность в 93%

### обучим все слои

In [ ]:
LR = 1e-5  # уменьшаем LR
model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])

# Загружаем веса для обученной части модели
model.load_weights('best_model.hdf5')

# Обучаем
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples//test_generator.batch_size,
    epochs=EPOCHS,
    callbacks=callbacks_list
)

# сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)
model.save('../working/model_last.hdf5')
model.load_weights('best_model.hdf5')

# сохраним лучшую модель для дальнейшего использования
model.save('../working/model_B6_LR_1e_5.hdf5')

scores = model.evaluate_generator(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

plot_history(history)

In [ ]:
def create_submission(model, f_name):
    """Функция для создания файла для submission"""
    test_sub_generator.reset()
    predictions = model.predict(
        test_sub_generator, steps=len(test_sub_generator), verbose=1)
    predictions = np.argmax(predictions, axis=-1)  # multiple categories
    label_map = (train_generator.class_indices)
    label_map = dict((v, k) for k, v in label_map.items())  # flip k,v
    predictions = [label_map[k] for k in predictions]

    filenames_with_dir = test_sub_generator.filenames
    submission = pd.DataFrame(
        {'Id': filenames_with_dir, 'Category': predictions}, columns=['Id', 'Category'])
    submission['Id'] = submission['Id'].replace('test_upload/', '')
    submission.to_csv(f_name, index=False)
    return 0

In [ ]:
create_submission(tf.keras.models.load_model(
    '../working/model_B6_LR_1e_5.hdf5'), 'submission_fine.csv')

### обучим еще одну модель с другими параметрами агументации и гиперпараметрами

In [ ]:
EPOCHS = 10  # эпох на обучение
BATCH_SIZE = 16
LR = 1e-5

AUGMENTATIONS = A.Compose([
    A.GaussianBlur(p=0.07),
    A.RandomBrightness(limit=0.3, p=0.6),
    A.ShiftScaleRotate(shift_limit=0.0625,
                       scale_limit=0.01,
                       interpolation=1,
                       border_mode=4,
                       rotate_limit=30,
                       p=.85),
    A.OneOf([
        A.CenterCrop(height=224, width=200),
        A.CenterCrop(height=200, width=224)], p=0.7),
    A.OneOf([
        A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
        A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)], p=0.3),
    A.HorizontalFlip(p=0.7),
    A.HueSaturationValue(p=0.3),
    A.RGBShift(p=0.8),
    A.FancyPCA(alpha=0.1, always_apply=False, p=0.3),
    A.Resize(IMG_SIZE, IMG_SIZE)
])

train_datagen = ImageDataAugmentor(rescale=1./255,
                                   augment=AUGMENTATIONS,
                                   seed=RANDOM_SEED,
                                   validation_split=VAL_SPLIT
                                   )

train_generator = train_datagen.flow_from_directory(
    PATH+'train/',      # директория где расположены папки с картинками
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training')  # set as training data

test_generator = train_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation')  # set as validation data

model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])
# Загрузим веса предыдущей обученной модели
model.load_weights('model_B6_LR_1e_5.hdf5')

# Обучаем
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples//test_generator.batch_size,
    epochs=EPOCHS,
    callbacks=callbacks_list
)

# сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)
model.save('../working/model_last.hdf5')
model.load_weights('best_model.hdf5')

# сохраним лучшую модель для дальнейшего использования
model.save('../working/model_B6_Other.hdf5')

scores = model.evaluate_generator(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

plot_history(history)

### Prediction after fine-tuning

In [ ]:
create_submission(tf.keras.models.load_model(
    '../working/model_B6_Other.hdf5'), 'submission_other.csv')

## IMG_SIZE rise

In [ ]:
# перезапустим код с новыми параметрами
EPOCHS = 7
BATCH_SIZE = 4
LR = 1e-5
IMG_SIZE = 512

# уменьшим число изменяемых параметров
AUGMENTATIONS = A.Compose([
    #A.RandomBrightness(limit=0.2, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.0625,
                       scale_limit=0.01,
                       interpolation=1,
                       border_mode=4,
                       rotate_limit=20,
                       p=.75),
    A.HorizontalFlip(p=0.5),
    # A.HueSaturationValue(p=0.5)
])

train_datagen = ImageDataAugmentor(rescale=1./255,
                                   augment=AUGMENTATIONS,
                                   seed=RANDOM_SEED,
                                   validation_split=VAL_SPLIT
                                   )

test_datagen = ImageDataAugmentor(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    PATH+'train/',      # директория где расположены папки с картинками
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training')  # set as training data

test_generator = train_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation')  # set as validation data

test_sub_generator = test_datagen.flow_from_dataframe(
    dataframe=sample_submission,
    directory=PATH+'test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,)

base_model = efn.EfficientNetB6(
    weights='imagenet', include_top=False, input_shape=input_shape)
base_model.trainable = True

model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])
model.load_weights('../working/model_B6_LR_1e_5.hdf5')

history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples//test_generator.batch_size,
    epochs=EPOCHS,
    callbacks=callbacks_list
)

# сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)
model.save('../working/model_last.hdf5')
model.load_weights('best_model.hdf5')

model.save('../working/model_img_size.hdf5')

scores = model.evaluate_generator(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

plot_history(history)

#### Prediction

In [ ]:
create_submission(tf.keras.models.load_model(
    '../working/model_img_size.hdf5'), 'submission_img_resize.csv')

In [ ]:
# перезапустим код с новыми параметрами
EPOCHS = 4
BATCH_SIZE = 2
LR = 1e-5
IMG_SIZE = 520

# уменьшим число изменяемых параметров
AUGMENTATIONS = A.Compose([
    A.RandomBrightness(limit=0.4, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.0625,
                       scale_limit=0.01,
                       interpolation=1,
                       border_mode=4,
                       rotate_limit=20,
                       p=.75),
    A.HorizontalFlip(p=0.5),
    A.HueSaturationValue(p=0.5)
])

train_datagen = ImageDataAugmentor(rescale=1./255,
                                   augment=AUGMENTATIONS,
                                   seed=RANDOM_SEED,
                                   validation_split=VAL_SPLIT
                                   )

test_datagen = ImageDataAugmentor(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    PATH+'train/',      # директория где расположены папки с картинками
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training')  # set as training data

test_generator = train_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation')  # set as validation data

test_sub_generator = test_datagen.flow_from_dataframe(
    dataframe=sample_submission,
    directory=PATH+'test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,)

base_model = efn.EfficientNetB6(
    weights='imagenet', include_top=False, input_shape=input_shape)
base_model.trainable = True

model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])
model.load_weights('../working/model_img_size.hdf5')

history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples//test_generator.batch_size,
    epochs=EPOCHS,
    callbacks=callbacks_list
)

# сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)
model.save('../working/model_last.hdf5')
model.load_weights('best_model.hdf5')

model.save('../working/model_img_size_v2.hdf5')

scores = model.evaluate_generator(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

plot_history(history)

# готовим submit
create_submission(tf.keras.models.load_model(
    '../working/model_img_size_v2.hdf5'), 'submission_img_resize_v2.csv')

## TTA

In [ ]:
# перезапустим код с новыми параметрами
BATCH_SIZE = 4
IMG_SIZE = 512

test_datagen = ImageDataAugmentor(rescale=1. / 255)

test_sub_generator = test_datagen.flow_from_dataframe(
    dataframe=sample_submission,
    directory=PATH+'test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,)

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])
model.load_weights('../working/model_img_size.hdf5')

tta_steps = 10
predictions = []

for i in range(tta_steps):
    preds = model.predict_generator(test_sub_generator, verbose=1)
    predictions.append(preds)

pred = np.mean(predictions, axis=0)

# Предсказание на тестовых данных

In [ ]:
predictions = np.argmax(pred, axis=-1)  # multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v, k) for k, v in label_map.items())  # flip k,v
predictions = [label_map[k] for k in predictions]

filenames_with_dir = test_sub_generator.filenames
submission = pd.DataFrame(
    {'Id': filenames_with_dir, 'Category': predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/', '')
submission.to_csv('submission_tta.csv', index=False)
print('Save submit')

In [ ]:
# Clean PATH
import shutil
shutil.rmtree(PATH)

# Итоги

В проекте применены:

- transfer learning и fine-tuning (обучение головы -> 50% разморозка весов предобученной модели -> 100% разморозка)
- дополнительные функции callback в Keras
- настройка LR
- настройка параметров регуляризации полносвязного слоя нейронной сети
- применен способ заполнения пропусков c помощью ImageDataAugmentor с использованием библиотеки аугментации изображений albumentations
- подобраны переменные (размер картинки, батч, количество эпох)
- добавлена Batch Normalization в архитектуре “головы” модели
- SOTA архитектура сетей - Xception, InceptionV3, EfficientNetB5
- добавлена TTA (Test Time Augmentation)


Ввиду ограничений на пользование ресурсами GPU и времязатратности самого просчета модели, не удалось протестировать следующие техники настройки модели:

- настройка optimizer
- настройка loss
- использование внешних датасетов для дообучения модели